# PRACTICA NLP. NOTEBOOK 3

## 3.- Etapa de entrenamiento y testeo de un modelo de análisis de sentimiento

El alumno, con los datos preprocesados del ejercicio 2, deberá entrenar dos modelos distintos de los que, tras comparar sus resultados, elegirá uno como el mejor.

Para tomar esta decisión se basará en las métricas que calcule (precision, recall, f1-score,...) 

El enfoque será el de un problema de clasificación binaria supervisada.

Los modelos deberán tomar a su entrada los datos codificados con un modelo de bolsa de palabras (bag-of-words). Se deberán justificar los parámetros del vectorizer, así como tener en cuenta aspectos como el balanceo de clases.

La elección de los modelos es libre.

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.pipeline import Pipeline
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
from test import *
import spacy
from spacy.tokens import Doc
from tqdm import tqdm
from typing import List
import pandas as pd

In [2]:
from ipynb.fs.defs.PRACTICA_NLP_2_Eva_Def import *
#from ipynb.fs.defs.PRACTICA_NLP_2_Eva_Def import text_cleaning
df = pd.read_csv('df_music_inst.csv', sep=';')

In [3]:
import cleaning_functions
import MyClass

In [4]:
#hago un subset para probar que funciona bien lo importado
df_revs= df['review'].tolist()[:100]

In [5]:
spacy_model = SpacyPreprocessor.load_model()
preprocessor = SpacyPreprocessor(spacy_model=spacy_model, lemmatize=True, remove_numbers=True, remove_stopwords=True, remove_special=True)
#clean_text = preprocessor.preprocess_text(df_rev[4])
clean_texts = preprocessor.preprocess_text_list(df_revs) 
print(clean_texts[:100])

100it [00:00, 104.54it/s]

['good beginner star', 'recommend starter ukulele kit need learn ukulele star', "g'daughter receive christmas present year play learn new song play regularly", 'accord order history buy kit end black friday week lightning deal write nearly year later stringed instrument string need stretch stay tune day play minute tune play minute tune tune daily week uke hold tune great tuner include package bulk minus star real beginner way tune thing need tuner reference note piano keyboard download decent tuner app buck go need lot couple week firebrand ukulele laminated instrument unknown wood kind dead sounding soprano uke generally sound bit flattish distinctive character finish smooth spotty decal sound hole bubble fretboard barely adequately finish fret nicely flush edge neck feel smooth stay smooth despite temperature change bevel fret edge file especially polished snag catch fret dot inset deeply fingerboard paint upper edge neck nice touch action okay near headstock starter chord live tune

In [6]:
#Del dataframe cargado que tenía 30000 reviews de cada tipo hago un subset de 5000 positivas y 5000 negativas. 
#una de las mejoras futuras será ampliar el dataset. 
df_pos=df[df.overall==1][:5000]
df_neg=df[df.overall==0][:5000]
df_music=df_pos.append(df_neg)

In [7]:
#preparo los datos para entrenar el modelo. X,y, train y test...
X= df_music['review']
y= df_music['overall']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [10]:
#compruebo los tamaños
X_train.shape, X_test.shape

((8000,), (2000,))

In [11]:
#y una muestra del resultado de la división
X_train[:20]

34254    Great string! BUT.... Gave two stars since 3 s...
1561            Good product. Works as expected Four Stars
1670             Tight pattern, but no feedback Five Stars
31087    Love the product! Keeps my strings sounding br...
31669    The odd thing to me is in the directions it sa...
30933    This capo is certainly cool.. Sadly, mine seem...
33829    bought 2, 1 didn't even last a week before brt...
32945    Decent for price. Don't sound quite as good as...
3508     A great sounding wah pedal. One of the best. A...
2002     We love these little Zenergy chimes! The sound...
30866    After reading dozens of reviews on these Prelu...
168      Works great with Yamaha StagePas systems for c...
2588     How do you review an SM58?? I've used these mo...
30905    These wedge are ok - typical foam, nothing mor...
34529    As a professional DJ I would not recommend thi...
31384    I tried this on a live gig from a buddy who ra...
32824    Since this is metal with no padding around the.

In [12]:
y_train.value_counts() #bastante balanceados los datos en la división

0.0    4012
1.0    3988
Name: overall, dtype: int64

In [13]:
# Hago pruebas con el TfIdfVectorizer, con el tokenizer utilizando la clase y las funciones importadas que he preparado de preprocesado:
#primero sin tocar el resto de parámetros del tfidf:
tfidf_default = TfidfVectorizer(tokenizer= preprocessor.preprocess_text_list)
X_train_tfidf_default = tfidf_default.fit_transform(X_train)
X_test_tfidf_default = tfidf_default.transform(X_test)


166it [00:00, 4878.46it/s]
42it [00:00, 3499.70it/s]
41it [00:00, 3725.04it/s]
314it [00:00, 5062.92it/s]
236it [00:00, 4916.68it/s]
303it [00:00, 5312.75it/s]
61it [00:00, 4062.31it/s]
155it [00:00, 5529.95it/s]
67it [00:00, 5565.49it/s]
1080it [00:00, 5998.73it/s]
1038it [00:00, 5966.42it/s]
89it [00:00, 4947.03it/s]
293it [00:00, 4801.04it/s]
297it [00:00, 5395.57it/s]
203it [00:00, 5480.67it/s]
765it [00:00, 5273.58it/s]
271it [00:00, 4839.33it/s]
217it [00:00, 5286.89it/s]
22it [00:00, 3143.73it/s]
440it [00:00, 5365.67it/s]
117it [00:00, 4681.19it/s]
197it [00:00, 5180.26it/s]
152it [00:00, 5624.18it/s]
27it [00:00, 3832.23it/s]
172it [00:00, 5925.28it/s]
144it [00:00, 5525.69it/s]
84it [00:00, 4929.99it/s]
27it [00:00, 3857.29it/s]
987it [00:00, 5944.34it/s]
126it [00:00, 5248.92it/s]
1120it [00:00, 5656.13it/s]
440it [00:00, 5866.77it/s]
154it [00:00, 5492.35it/s]
1322it [00:00, 4542.48it/s]
2775it [00:00, 5854.32it/s]
78it [00:00, 4598.57it/s]
227it [00:00, 5668.21it/s]
23it [

239it [00:00, 5775.75it/s]
96it [00:00, 5633.64it/s]
252it [00:00, 6000.43it/s]
642it [00:00, 5783.83it/s]
157it [00:00, 6038.68it/s]
632it [00:00, 5543.82it/s]
38it [00:00, 4735.67it/s]
54it [00:00, 4488.91it/s]
174it [00:00, 5602.71it/s]
23it [00:00, 3278.47it/s]
454it [00:00, 6050.83it/s]
187it [00:00, 4791.76it/s]
403it [00:00, 6196.31it/s]
34it [00:00, 4235.29it/s]
564it [00:00, 6064.30it/s]
210it [00:00, 5379.32it/s]
501it [00:00, 5964.58it/s]
39it [00:00, 3545.71it/s]
173it [00:00, 5970.51it/s]
497it [00:00, 6369.41it/s]
141it [00:00, 5632.89it/s]
55it [00:00, 4582.94it/s]
155it [00:00, 5736.19it/s]
1080it [00:00, 4500.04it/s]
696it [00:00, 6212.10it/s]
147it [00:00, 5444.41it/s]
419it [00:00, 5983.74it/s]
102it [00:00, 5085.70it/s]
16it [00:00, 2649.28it/s]
133it [00:00, 5543.50it/s]
97it [00:00, 5692.72it/s]
53it [00:00, 5286.90it/s]
23it [00:00, 3277.69it/s]
274it [00:00, 6086.43it/s]
204it [00:00, 6176.15it/s]
223it [00:00, 5720.36it/s]
481it [00:00, 6090.30it/s]
192it [00:0

1815it [00:00, 5797.97it/s]
247it [00:00, 6018.32it/s]
414it [00:00, 5828.55it/s]
146it [00:00, 6083.29it/s]
3028it [00:00, 5445.15it/s]
297it [00:00, 5496.08it/s]
220it [00:00, 5636.64it/s]
457it [00:00, 5858.56it/s]
289it [00:00, 5554.66it/s]
94it [00:00, 5210.59it/s]
638it [00:00, 6100.83it/s]
45it [00:00, 4483.54it/s]
103it [00:00, 5411.33it/s]
262it [00:00, 4854.90it/s]
473it [00:00, 5839.45it/s]
238it [00:00, 5661.68it/s]
303it [00:00, 5941.44it/s]
515it [00:00, 6060.15it/s]
105it [00:00, 4764.45it/s]
1074it [00:00, 5478.78it/s]
811it [00:00, 5442.02it/s]
196it [00:00, 5444.73it/s]
28it [00:00, 3487.88it/s]
4257it [00:00, 5940.90it/s]
362it [00:00, 5694.79it/s]
86it [00:00, 4777.36it/s]
63it [00:00, 4833.74it/s]
64it [00:00, 4556.55it/s]
317it [00:00, 5660.06it/s]
117it [00:00, 5838.31it/s]
154it [00:00, 5489.97it/s]
15it [00:00, 2982.72it/s]
16it [00:00, 2285.57it/s]
101it [00:00, 5614.79it/s]
54it [00:00, 4925.25it/s]
669it [00:00, 6077.27it/s]
326it [00:00, 5425.30it/s]
411it 

437it [00:00, 5675.49it/s]
624it [00:00, 4950.14it/s]
32it [00:00, 3202.52it/s]
190it [00:00, 4634.37it/s]
68it [00:00, 4532.65it/s]
17it [00:00, 2832.75it/s]
113it [00:00, 5128.95it/s]
187it [00:00, 5659.84it/s]
40it [00:00, 4001.63it/s]
168it [00:00, 5796.19it/s]
23it [00:00, 2875.98it/s]
817it [00:00, 5409.25it/s]
413it [00:00, 5814.20it/s]
48it [00:00, 4355.08it/s]
316it [00:00, 5743.63it/s]
72it [00:00, 5526.70it/s]
324it [00:00, 5680.37it/s]
414it [00:00, 5075.86it/s]
98it [00:00, 4891.14it/s]
45it [00:00, 4483.01it/s]
150it [00:00, 4838.13it/s]
13it [00:00, 2168.55it/s]
99it [00:00, 4296.06it/s]
21it [00:00, 2332.88it/s]
394it [00:00, 5877.30it/s]
124it [00:00, 5626.16it/s]
29it [00:00, 4120.00it/s]
143it [00:00, 5705.88it/s]
57it [00:00, 3562.49it/s]
766it [00:00, 6079.10it/s]
1398it [00:00, 5257.50it/s]
7it [00:00, 1400.03it/s]
628it [00:00, 5925.49it/s]
133it [00:00, 6035.69it/s]
1359it [00:00, 4261.96it/s]
21it [00:00, 3001.04it/s]
175it [00:00, 5836.54it/s]
651it [00:00, 57

171it [00:00, 5345.21it/s]
2536it [00:00, 5995.37it/s]
144it [00:00, 6004.55it/s]
1620it [00:00, 6090.28it/s]
573it [00:00, 5457.15it/s]
143it [00:00, 5296.63it/s]
327it [00:00, 5450.11it/s]
151it [00:00, 5207.02it/s]
357it [00:00, 5666.48it/s]
1133it [00:00, 6293.02it/s]
606it [00:00, 6183.44it/s]
581it [00:00, 5480.88it/s]
102it [00:00, 5369.21it/s]
1465it [00:00, 6053.75it/s]
106it [00:00, 5890.02it/s]
173it [00:00, 5766.95it/s]
61it [00:00, 5084.00it/s]
627it [00:00, 6087.40it/s]
102it [00:00, 5099.03it/s]
723it [00:00, 6025.11it/s]
110it [00:00, 5235.68it/s]
127it [00:00, 6048.06it/s]
29it [00:00, 3624.61it/s]
162it [00:00, 6000.75it/s]
673it [00:00, 5903.79it/s]
43it [00:00, 4034.07it/s]
408it [00:00, 6276.99it/s]
73it [00:00, 4853.44it/s]
420it [00:00, 5675.68it/s]
103it [00:00, 5724.61it/s]
730it [00:00, 5983.15it/s]
2846it [00:00, 5420.95it/s]
224it [00:00, 5600.71it/s]
21it [00:00, 2983.65it/s]
27it [00:00, 4473.13it/s]
685it [00:00, 6116.14it/s]
697it [00:00, 6114.53it/s]
19

...

198it [00:00, 5651.82it/s]
62it [00:00, 4770.98it/s]
19it [00:00, 3165.26it/s]
74it [00:00, 4934.48it/s]
2469it [00:00, 5864.64it/s]
251it [00:00, 5975.67it/s]
65it [00:00, 4642.72it/s]
153it [00:00, 5661.73it/s]
190it [00:00, 5581.75it/s]
592it [00:00, 5381.63it/s]
186it [00:00, 5806.30it/s]
334it [00:00, 5758.92it/s]
192it [00:00, 5812.39it/s]
224it [00:00, 5601.71it/s]
230it [00:00, 5471.80it/s]
458it [00:00, 5325.67it/s]
246it [00:00, 5716.59it/s]
57it [00:00, 4748.83it/s]
46it [00:00, 4600.55it/s]
84it [00:00, 5250.22it/s]
298it [00:00, 5623.50it/s]
28it [00:00, 3999.74it/s]
26it [00:00, 3695.55it/s]
114it [00:00, 5428.84it/s]
43it [00:00, 4285.80it/s]
34it [00:00, 4249.04it/s]
41it [00:00, 4538.93it/s]
110it [00:00, 5509.13it/s]
657it [00:00, 5763.37it/s]
134it [00:00, 5583.68it/s]
229it [00:00, 5452.81it/s]
164it [00:00, 5284.53it/s]
505it [00:00, 5674.22it/s]
110it [00:00, 956.23it/s]
34it [00:00, 4225.75it/s]
2022it [00:00, 5929.02it/s]
138it [00:00, 5998.82it/s]
55it [00:00, 

188it [00:00, 5690.43it/s]
206it [00:00, 5562.02it/s]
676it [00:00, 5927.92it/s]
38it [00:00, 4730.04it/s]
974it [00:00, 6012.33it/s]
336it [00:00, 5695.95it/s]
175it [00:00, 5462.27it/s]
929it [00:00, 5841.76it/s]
546it [00:00, 5569.34it/s]
372it [00:00, 5467.74it/s]
660it [00:00, 5411.41it/s]
205it [00:00, 5694.74it/s]
284it [00:00, 5563.62it/s]
151it [00:00, 6026.99it/s]
367it [00:00, 5825.47it/s]
998it [00:00, 5801.03it/s]
88it [00:00, 5500.40it/s]
127it [00:00, 2956.47it/s]
108it [00:00, 4901.05it/s]
103it [00:00, 5149.76it/s]
72it [00:00, 4798.97it/s]
35it [00:00, 4376.88it/s]
93it [00:00, 5168.34it/s]
538it [00:00, 5660.93it/s]
25it [00:00, 3572.54it/s]
502it [00:00, 5702.32it/s]
166it [00:00, 5465.19it/s]
72it [00:00, 5123.25it/s]
194it [00:00, 5536.81it/s]
765it [00:00, 5974.69it/s]
44it [00:00, 4401.58it/s]
147it [00:00, 5879.25it/s]
331it [00:00, 5703.49it/s]
713it [00:00, 3411.41it/s]
247it [00:00, 5141.86it/s]
1310it [00:00, 5796.47it/s]
43it [00:00, 4754.19it/s]
878it [00

263it [00:00, 4958.45it/s]
100it [00:00, 4990.72it/s]
493it [00:00, 5666.77it/s]
322it [00:00, 5547.93it/s]
379it [00:00, 5918.69it/s]
308it [00:00, 5310.03it/s]
758it [00:00, 5874.01it/s]
716it [00:00, 6117.52it/s]
198it [00:00, 5824.11it/s]
176it [00:00, 5500.15it/s]
232it [00:00, 5948.89it/s]
596it [00:00, 6079.35it/s]
373it [00:00, 5917.26it/s]
636it [00:00, 5834.88it/s]
332it [00:00, 5626.91it/s]
1105it [00:00, 5904.90it/s]
233it [00:00, 5824.94it/s]
494it [00:00, 4489.35it/s]
909it [00:00, 5575.41it/s]
29it [00:00, 3624.61it/s]
2126it [00:00, 5823.84it/s]
20it [00:00, 3314.08it/s]
1167it [00:00, 5719.57it/s]
62it [00:00, 5184.66it/s]
227it [00:00, 5669.60it/s]
1035it [00:00, 5813.38it/s]
667it [00:00, 5652.42it/s]
246it [00:00, 5851.42it/s]
185it [00:00, 5279.48it/s]
549it [00:00, 5778.86it/s]
27it [00:00, 3375.14it/s]
1407it [00:00, 5838.24it/s]
267it [00:00, 5620.16it/s]
59it [00:00, 4902.61it/s]
145it [00:00, 5791.14it/s]
176it [00:00, 5671.07it/s]
631it [00:00, 5684.69it/s]
6

40it [00:00, 4444.06it/s]
336it [00:00, 5895.31it/s]
79it [00:00, 5632.04it/s]
87it [00:00, 4834.00it/s]
21it [00:00, 3480.20it/s]
169it [00:00, 5626.01it/s]
165it [00:00, 5494.63it/s]
23it [00:00, 3811.35it/s]
84it [00:00, 5587.79it/s]
23it [00:00, 3810.29it/s]
35it [00:00, 4373.75it/s]
56it [00:00, 4656.46it/s]
38it [00:00, 3787.00it/s]
402it [00:00, 5504.08it/s]
1348it [00:00, 5570.17it/s]
209it [00:00, 4856.13it/s]
409it [00:00, 5787.24it/s]
531it [00:00, 5528.73it/s]
184it [00:00, 5568.80it/s]
192it [00:00, 5639.83it/s]
372it [00:00, 4828.96it/s]
32it [00:00, 3977.65it/s]
558it [00:00, 5871.40it/s]
318it [00:00, 5386.15it/s]
1167it [00:00, 5954.10it/s]
94it [00:00, 5530.20it/s]
279it [00:00, 5812.72it/s]
160it [00:00, 5709.01it/s]
315it [00:00, 2099.46it/s]
606it [00:00, 5826.85it/s]
29it [00:00, 4145.13it/s]
251it [00:00, 5971.10it/s]
631it [00:00, 6067.39it/s]
54it [00:00, 4500.24it/s]
15it [00:00, 2979.19it/s]
2029it [00:00, 5731.11it/s]
128it [00:00, 6088.28it/s]
2110it [00:00

1551it [00:00, 5538.78it/s]
123it [00:00, 5118.30it/s]
1331it [00:00, 5836.71it/s]
876it [00:00, 6039.89it/s]
81it [00:00, 5399.36it/s]
665it [00:00, 6100.80it/s]
208it [00:00, 5937.10it/s]
340it [00:00, 5961.11it/s]
35it [00:00, 4395.76it/s]
169it [00:00, 6035.84it/s]
290it [00:00, 4676.55it/s]
908it [00:00, 5815.78it/s]
503it [00:00, 6087.33it/s]
140it [00:00, 6077.82it/s]
156it [00:00, 5991.37it/s]
119it [00:00, 5408.31it/s]
193it [00:00, 4733.09it/s]
482it [00:00, 6098.01it/s]
334it [00:00, 5960.33it/s]
192it [00:00, 5485.81it/s]
157it [00:00, 5607.41it/s]
39it [00:00, 4857.98it/s]
131it [00:00, 5459.05it/s]
228it [00:00, 5700.11it/s]
44it [00:00, 4401.05it/s]
15it [00:00, 3000.93it/s]
60it [00:00, 4985.11it/s]
558it [00:00, 5634.25it/s]
744it [00:00, 5677.66it/s]
1489it [00:00, 5885.37it/s]
135it [00:00, 5860.57it/s]
135it [00:00, 5869.81it/s]
31it [00:00, 4428.89it/s]
592it [00:00, 6038.24it/s]
92it [00:00, 5412.16it/s]
390it [00:00, 5732.17it/s]
58it [00:00, 5254.77it/s]
259it [

In [14]:
#después probando con bigramas y con un limite superior e inferior de frecuencia de palabras.
tfidf_cust = TfidfVectorizer(tokenizer= preprocessor.preprocess_text_list,ngram_range=(1, 2), max_df=0.97, min_df=0.5)
X_train_tfidf_cust = tfidf_cust.fit_transform(X_train)
X_test_tfidf_cust = tfidf_cust.transform(X_test)

166it [00:00, 5030.99it/s]
42it [00:00, 3230.71it/s]
41it [00:00, 3728.35it/s]
314it [00:00, 5233.57it/s]
236it [00:00, 5488.77it/s]
303it [00:00, 5180.60it/s]
61it [00:00, 5067.69it/s]
155it [00:00, 5338.76it/s]
67it [00:00, 5155.64it/s]
1080it [00:00, 5453.60it/s]
1038it [00:00, 5765.53it/s]
89it [00:00, 5563.65it/s]
293it [00:00, 5524.78it/s]
297it [00:00, 4935.04it/s]
203it [00:00, 5205.31it/s]
765it [00:00, 5839.45it/s]
271it [00:00, 5526.07it/s]
217it [00:00, 4927.96it/s]
22it [00:00, 3690.99it/s]
440it [00:00, 5711.66it/s]
117it [00:00, 5572.09it/s]
197it [00:00, 5306.31it/s]
152it [00:00, 5629.59it/s]
27it [00:00, 3834.56it/s]
172it [00:00, 5548.23it/s]
144it [00:00, 5751.75it/s]
84it [00:00, 5586.55it/s]
27it [00:00, 3834.95it/s]
987it [00:00, 5750.70it/s]
126it [00:00, 5990.23it/s]
1120it [00:00, 5989.40it/s]
440it [00:00, 5789.69it/s]
154it [00:00, 5923.61it/s]
1322it [00:00, 5901.68it/s]
2775it [00:00, 5942.21it/s]
78it [00:00, 5200.13it/s]
227it [00:00, 5675.34it/s]
23it [

239it [00:00, 5432.01it/s]
96it [00:00, 5647.71it/s]
252it [00:00, 5727.44it/s]
642it [00:00, 5836.41it/s]
157it [00:00, 6280.46it/s]
632it [00:00, 5906.63it/s]
38it [00:00, 4223.20it/s]
54it [00:00, 4909.45it/s]
174it [00:00, 5612.84it/s]
23it [00:00, 3834.83it/s]
454it [00:00, 5896.29it/s]
187it [00:00, 6233.64it/s]
403it [00:00, 6015.10it/s]
34it [00:00, 4227.00it/s]
564it [00:00, 5936.97it/s]
210it [00:00, 5828.20it/s]
501it [00:00, 5823.07it/s]
39it [00:00, 4319.80it/s]
173it [00:00, 5965.85it/s]
497it [00:00, 5779.25it/s]
141it [00:00, 5035.69it/s]
55it [00:00, 3928.99it/s]
155it [00:00, 5167.41it/s]
1080it [00:00, 5427.23it/s]
696it [00:00, 5568.35it/s]
147it [00:00, 4900.32it/s]
419it [00:00, 5739.79it/s]
102it [00:00, 5100.25it/s]
16it [00:00, 2666.22it/s]
133it [00:00, 5783.45it/s]
97it [00:00, 5389.14it/s]
53it [00:00, 4417.43it/s]
23it [00:00, 3286.74it/s]
274it [00:00, 5592.38it/s]
204it [00:00, 5587.84it/s]
223it [00:00, 5439.26it/s]
481it [00:00, 5593.01it/s]
192it [00:0

1815it [00:00, 6153.04it/s]
247it [00:00, 5613.65it/s]
414it [00:00, 6088.46it/s]
146it [00:00, 6083.47it/s]
3028it [00:00, 6036.01it/s]
297it [00:00, 5400.16it/s]
220it [00:00, 6278.21it/s]
457it [00:00, 5855.98it/s]
289it [00:00, 5066.90it/s]
94it [00:00, 5530.12it/s]
638it [00:00, 5904.92it/s]
45it [00:00, 5000.49it/s]
103it [00:00, 5722.56it/s]
262it [00:00, 5822.03it/s]
473it [00:00, 6139.79it/s]
238it [00:00, 5944.27it/s]
303it [00:00, 5941.80it/s]
515it [00:00, 5659.35it/s]
105it [00:00, 5526.16it/s]
1074it [00:00, 6102.26it/s]
811it [00:00, 6068.97it/s]
196it [00:00, 5603.00it/s]
28it [00:00, 3484.99it/s]
4257it [00:00, 5255.47it/s]
362it [00:00, 5838.80it/s]
86it [00:00, 5374.67it/s]
63it [00:00, 5251.01it/s]
64it [00:00, 4923.07it/s]
317it [00:00, 5196.98it/s]
117it [00:00, 5574.36it/s]
154it [00:00, 5502.93it/s]
15it [00:00, 3000.07it/s]
16it [00:00, 2285.57it/s]
101it [00:00, 5051.15it/s]
54it [00:00, 4500.59it/s]
669it [00:00, 6026.84it/s]
326it [00:00, 5821.26it/s]
411it 

437it [00:00, 5200.12it/s]
624it [00:00, 5831.70it/s]
32it [00:00, 4570.36it/s]
190it [00:00, 5751.14it/s]
68it [00:00, 4848.25it/s]
17it [00:00, 2835.57it/s]
113it [00:00, 5639.99it/s]
187it [00:00, 5053.67it/s]
40it [00:00, 4001.15it/s]
168it [00:00, 5592.76it/s]
23it [00:00, 3283.27it/s]
817it [00:00, 5752.05it/s]
413it [00:00, 5817.11it/s]
48it [00:00, 4780.51it/s]
316it [00:00, 5639.57it/s]
72it [00:00, 5127.60it/s]
324it [00:00, 5764.44it/s]
414it [00:00, 5910.83it/s]
98it [00:00, 5148.00it/s]
45it [00:00, 4482.26it/s]
150it [00:00, 5768.91it/s]
13it [00:00, 2165.45it/s]
99it [00:00, 4950.12it/s]
21it [00:00, 3507.36it/s]
394it [00:00, 5394.59it/s]
124it [00:00, 5391.14it/s]
29it [00:00, 4001.28it/s]
143it [00:00, 5492.39it/s]
57it [00:00, 5163.50it/s]
766it [00:00, 4877.89it/s]
1398it [00:00, 5800.00it/s]
7it [00:00, 1387.46it/s]
628it [00:00, 5978.64it/s]
133it [00:00, 5541.30it/s]
1359it [00:00, 6012.35it/s]
21it [00:00, 3477.86it/s]
175it [00:00, 5645.40it/s]
651it [00:00, 59

171it [00:00, 6107.33it/s]
2536it [00:00, 4766.52it/s]
144it [00:00, 5751.42it/s]
1620it [00:00, 5765.21it/s]
573it [00:00, 5969.17it/s]
143it [00:00, 5296.31it/s]
327it [00:00, 5736.99it/s]
151it [00:00, 6040.50it/s]
357it [00:00, 5754.68it/s]
1133it [00:00, 5581.91it/s]
606it [00:00, 5877.28it/s]
581it [00:00, 5752.55it/s]
102it [00:00, 5367.73it/s]
1465it [00:00, 5907.38it/s]
106it [00:00, 5579.28it/s]
173it [00:00, 5406.64it/s]
61it [00:00, 4692.65it/s]
627it [00:00, 5805.45it/s]
102it [00:00, 5369.15it/s]
723it [00:00, 5878.34it/s]
110it [00:00, 5238.47it/s]
127it [00:00, 5772.46it/s]
29it [00:00, 3625.15it/s]
162it [00:00, 5786.33it/s]
673it [00:00, 6009.10it/s]
43it [00:00, 3910.05it/s]
408it [00:00, 4340.40it/s]
73it [00:00, 3042.22it/s]
420it [00:00, 4421.24it/s]
103it [00:00, 4120.10it/s]
730it [00:00, 5140.82it/s]
2846it [00:00, 5737.92it/s]
224it [00:00, 5333.51it/s]
21it [00:00, 3524.34it/s]
27it [00:00, 3838.07it/s]
685it [00:00, 5905.20it/s]
697it [00:00, 5857.30it/s]
19

...

157it [00:00, 6270.11it/s]
143it [00:00, 6217.46it/s]
48it [00:00, 4783.92it/s]
28it [00:00, 4636.42it/s]
149it [00:00, 6466.73it/s]
867it [00:00, 5856.43it/s]
239it [00:00, 5968.78it/s]
17it [00:00, 3400.57it/s]
256it [00:00, 5015.47it/s]
435it [00:00, 6038.41it/s]
385it [00:00, 6015.30it/s]
116it [00:00, 5800.42it/s]
323it [00:00, 5471.10it/s]
26it [00:00, 4306.10it/s]
631it [00:00, 5894.79it/s]
146it [00:00, 6083.83it/s]
630it [00:00, 6114.09it/s]
459it [00:00, 5733.78it/s]
89it [00:00, 5558.26it/s]
418it [00:00, 5570.55it/s]
116it [00:00, 5800.14it/s]
3753it [00:00, 6043.06it/s]
623it [00:00, 6105.55it/s]
66it [00:00, 5501.05it/s]
512it [00:00, 6240.74it/s]
817it [00:00, 6050.22it/s]
193it [00:00, 6023.52it/s]
377it [00:00, 5623.41it/s]
82it [00:00, 5840.95it/s]
113it [00:00, 6264.95it/s]
99it [00:00, 5824.44it/s]
58it [00:00, 4818.08it/s]
423it [00:00, 6041.85it/s]
198it [00:00, 5994.80it/s]
287it [00:00, 5852.41it/s]
402it [00:00, 6277.57it/s]
207it [00:00, 6264.54it/s]
203it [00

262it [00:00, 5811.16it/s]
20it [00:00, 3319.46it/s]
242it [00:00, 6046.55it/s]
774it [00:00, 5862.58it/s]
220it [00:00, 5636.20it/s]
350it [00:00, 6025.43it/s]
58it [00:00, 5255.23it/s]
37it [00:00, 4609.26it/s]
506it [00:00, 6020.06it/s]
421it [00:00, 6098.20it/s]
1227it [00:00, 6117.20it/s]
165it [00:00, 6346.32it/s]
963it [00:00, 6018.74it/s]
33it [00:00, 4693.05it/s]
72it [00:00, 5584.86it/s]
126it [00:00, 5992.00it/s]
140it [00:00, 5825.54it/s]
170it [00:00, 5861.96it/s]
495it [00:00, 5956.18it/s]
241it [00:00, 6173.03it/s]
46it [00:00, 4586.23it/s]
39it [00:00, 4333.99it/s]
589it [00:00, 6010.36it/s]
27it [00:00, 4471.89it/s]
37it [00:00, 5284.12it/s]
141it [00:00, 6130.50it/s]
558it [00:00, 5936.18it/s]
302it [00:00, 6030.63it/s]
451it [00:00, 6172.14it/s]
37it [00:00, 4626.58it/s]
104it [00:00, 6104.56it/s]
442it [00:00, 5020.45it/s]
307it [00:00, 6137.25it/s]
157it [00:00, 6531.05it/s]
330it [00:00, 5999.81it/s]
159it [00:00, 6363.07it/s]
350it [00:00, 6250.13it/s]
225it [00:

198it [00:00, 5999.96it/s]
62it [00:00, 5636.77it/s]
19it [00:00, 3799.73it/s]
74it [00:00, 5692.62it/s]
2469it [00:00, 5892.60it/s]
251it [00:00, 5976.35it/s]
65it [00:00, 5416.95it/s]
153it [00:00, 6375.27it/s]
190it [00:00, 6129.19it/s]
592it [00:00, 5803.94it/s]
186it [00:00, 6000.20it/s]
334it [00:00, 5964.29it/s]
192it [00:00, 6000.21it/s]
224it [00:00, 6054.10it/s]
230it [00:00, 5897.50it/s]
458it [00:00, 6106.80it/s]
246it [00:00, 5466.89it/s]
57it [00:00, 5178.04it/s]
46it [00:00, 5115.82it/s]
84it [00:00, 5253.90it/s]
298it [00:00, 5843.23it/s]
28it [00:00, 4667.93it/s]
26it [00:00, 4333.48it/s]
114it [00:00, 5182.14it/s]
43it [00:00, 5375.07it/s]
34it [00:00, 4855.84it/s]
41it [00:00, 5124.46it/s]
110it [00:00, 6111.23it/s]
657it [00:00, 5918.92it/s]
134it [00:00, 5583.18it/s]
229it [00:00, 5724.70it/s]
164it [00:00, 5857.52it/s]
505it [00:00, 6085.15it/s]
110it [00:00, 6115.69it/s]
34it [00:00, 4855.34it/s]
2022it [00:00, 6202.47it/s]
138it [00:00, 6269.17it/s]
55it [00:00,

188it [00:00, 4372.09it/s]
206it [00:00, 6237.02it/s]
676it [00:00, 6087.72it/s]
38it [00:00, 5395.70it/s]
974it [00:00, 6012.44it/s]
336it [00:00, 5505.30it/s]
175it [00:00, 5147.14it/s]
929it [00:00, 4914.58it/s]
546it [00:00, 5351.07it/s]
372it [00:00, 6413.76it/s]
660it [00:00, 6295.62it/s]
205it [00:00, 6212.21it/s]
284it [00:00, 5916.80it/s]
151it [00:00, 6292.00it/s]
367it [00:00, 6016.34it/s]
998it [00:00, 6048.46it/s]
88it [00:00, 5501.05it/s]
127it [00:00, 5784.93it/s]
108it [00:00, 5674.87it/s]
103it [00:00, 5721.50it/s]
72it [00:00, 5538.56it/s]
35it [00:00, 4999.00it/s]
93it [00:00, 4894.48it/s]
538it [00:00, 6210.19it/s]
25it [00:00, 4163.82it/s]
502it [00:00, 6119.87it/s]
166it [00:00, 5919.83it/s]
72it [00:00, 5537.44it/s]
194it [00:00, 5878.58it/s]
765it [00:00, 5836.87it/s]
44it [00:00, 4399.58it/s]
147it [00:00, 5645.76it/s]
331it [00:00, 6360.45it/s]
713it [00:00, 5991.56it/s]
247it [00:00, 6024.30it/s]
1310it [00:00, 5796.46it/s]
43it [00:00, 4776.73it/s]
878it [00

263it [00:00, 5844.16it/s]
100it [00:00, 5882.28it/s]
493it [00:00, 5869.11it/s]
322it [00:00, 5962.47it/s]
379it [00:00, 6112.76it/s]
308it [00:00, 6038.19it/s]
758it [00:00, 6015.88it/s]
716it [00:00, 6226.10it/s]
198it [00:00, 6187.49it/s]
176it [00:00, 5949.13it/s]
232it [00:00, 6105.28it/s]
596it [00:00, 6116.96it/s]
373it [00:00, 6222.11it/s]
636it [00:00, 6167.02it/s]
332it [00:00, 6143.55it/s]
1105it [00:00, 6137.03it/s]
233it [00:00, 6125.11it/s]
494it [00:00, 6171.90it/s]
909it [00:00, 6140.75it/s]
29it [00:00, 4804.66it/s]
2126it [00:00, 6022.18it/s]
20it [00:00, 3977.53it/s]
1167it [00:00, 6142.60it/s]
62it [00:00, 5616.81it/s]
227it [00:00, 5674.97it/s]
1035it [00:00, 5913.74it/s]
667it [00:00, 6117.22it/s]
246it [00:00, 6150.56it/s]
185it [00:00, 5774.44it/s]
549it [00:00, 6095.18it/s]
27it [00:00, 4468.54it/s]
1407it [00:00, 6063.18it/s]
267it [00:00, 5930.28it/s]
59it [00:00, 5878.14it/s]
145it [00:00, 6293.39it/s]
176it [00:00, 5857.18it/s]
631it [00:00, 6126.30it/s]
6

In [15]:
lr_tfidf_default = LogisticRegression() #pendiente modificar parámetros en la regresión logística
lr_tfidf_cust = LogisticRegression()

In [16]:
lr_tfidf_default.fit(X_train_tfidf_default, y_train)  # train LR parámetros por defecto del tfidf
y_pred_tfidf_default = lr_tfidf_default.predict(X_test_tfidf_default)  # test LR

In [17]:
lr_tfidf_cust.fit(X_train_tfidf_cust, y_train)  # train LR parámetros tfidf retocados 
y_pred_tfidf_cust = lr_tfidf_cust.predict(X_test_tfidf_cust)  # test

In [18]:
print('Logistic regression TfIdfVectorizer default\n')
print(confusion_matrix(y_test, y_pred_tfidf_default))
print(classification_report(y_test, y_pred_tfidf_default))
#los valores por defecto no son muy buenos... la muestra está balanceada, en la matriz de confusión vemos que el modelo acierta mejor en los negativos que en los positivos. 

Logistic regression TfIdfVectorizer default

[[774 214]
 [404 608]]
              precision    recall  f1-score   support

         0.0       0.66      0.78      0.71       988
         1.0       0.74      0.60      0.66      1012

    accuracy                           0.69      2000
   macro avg       0.70      0.69      0.69      2000
weighted avg       0.70      0.69      0.69      2000



In [19]:
print('Logistic Regression TfIdfVectorizer customized\n')
print(confusion_matrix(y_test, y_pred_tfidf_cust))
print(classification_report(y_test, y_pred_tfidf_cust))
#añadiendo bigramas y limitando por arriba y por abajo las palabras por frecuencia parece que mejora algo. 
#podría hacer más ajustes pero pruebo primero con otros modelos. 

Logistic Regression TfIdfVectorizer customized

[[801 187]
 [322 690]]
              precision    recall  f1-score   support

         0.0       0.71      0.81      0.76       988
         1.0       0.79      0.68      0.73      1012

    accuracy                           0.75      2000
   macro avg       0.75      0.75      0.74      2000
weighted avg       0.75      0.75      0.74      2000



In [20]:
#pruebo con random forest con los mismos parámetros en el tfidf, por defecto y retocados.
rf_tfidf_default = RandomForestClassifier(n_estimators=100,n_jobs=-1)
rf_tfidf_cust = RandomForestClassifier(n_estimators=100,n_jobs=-1)

In [21]:
rf_tfidf_default.fit(X_train_tfidf_default, y_train)  # train RF tfidf default
rf_y_pred_tfidf_default = rf_tfidf_default.predict(X_test_tfidf_default)  # test

In [22]:
rf_tfidf_cust.fit(X_train_tfidf_cust, y_train)  # train RF tfidf retocados
rf_y_pred_tfidf_cust = rf_tfidf_cust.predict(X_test_tfidf_cust)  # test

In [23]:
print('Random forest TfIdfVectorizer default\n')
print(confusion_matrix(y_test, rf_y_pred_tfidf_default))
print(classification_report(y_test, rf_y_pred_tfidf_default))
#algo mejora con random forest 

Random forest TfIdfVectorizer default

[[785 203]
 [349 663]]
              precision    recall  f1-score   support

         0.0       0.69      0.79      0.74       988
         1.0       0.77      0.66      0.71      1012

    accuracy                           0.72      2000
   macro avg       0.73      0.72      0.72      2000
weighted avg       0.73      0.72      0.72      2000



In [24]:
print('Random Forest TfIdfVectorizer customized\n')
print(confusion_matrix(y_test, rf_y_pred_tfidf_cust))
print(classification_report(y_test, rf_y_pred_tfidf_cust))
#Mejora algo más, antes de retocar voy a hacer una última prueba con LinearSVC. 

Random Forest TfIdfVectorizer customized

[[804 184]
 [276 736]]
              precision    recall  f1-score   support

         0.0       0.74      0.81      0.78       988
         1.0       0.80      0.73      0.76      1012

    accuracy                           0.77      2000
   macro avg       0.77      0.77      0.77      2000
weighted avg       0.77      0.77      0.77      2000



In [25]:
SVC_tfidf_default = LinearSVC(C=100,max_iter=10000) 
SVC_tfidf_cust = LinearSVC(C=100,max_iter=10000) 
#pruebo con C=a 100 porque he visto en algún ejemplo similar que funciona mejor que 1. Me pide que suba el número de iteraciones para que converja y lo subo. si no mejora cambio. 

In [26]:
SVC_tfidf_default.fit(X_train_tfidf_default, y_train)  # train SVC default en tfidf
SVC_y_pred_tfidf_default = SVC_tfidf_default.predict(X_test_tfidf_default)  # test

In [27]:
SVC_tfidf_cust.fit(X_train_tfidf_cust, y_train)  # train SVC param retocados en tfidf
SVC_y_pred_tfidf_cust = SVC_tfidf_cust.predict(X_test_tfidf_cust)  # test

In [28]:
print('SVC TfIdfVectorizer default\n')
print(confusion_matrix(y_test, SVC_y_pred_tfidf_default))
print(classification_report(y_test, SVC_y_pred_tfidf_default))
#por poco parece que el mejor resultado es el del random forest en versión tfidf retocado. 

SVC TfIdfVectorizer default

[[775 213]
 [405 607]]
              precision    recall  f1-score   support

         0.0       0.66      0.78      0.71       988
         1.0       0.74      0.60      0.66      1012

    accuracy                           0.69      2000
   macro avg       0.70      0.69      0.69      2000
weighted avg       0.70      0.69      0.69      2000



In [29]:
print('SVC TfIdfVectorizer customized\n')
print(confusion_matrix(y_test, SVC_y_pred_tfidf_cust))
print(classification_report(y_test, SVC_y_pred_tfidf_cust))
# el svm no mejora los resultados del random forest, claro que habría que retocar parámetros en todos. 

SVC TfIdfVectorizer customized

[[792 196]
 [301 711]]
              precision    recall  f1-score   support

         0.0       0.72      0.80      0.76       988
         1.0       0.78      0.70      0.74      1012

    accuracy                           0.75      2000
   macro avg       0.75      0.75      0.75      2000
weighted avg       0.75      0.75      0.75      2000



In [30]:
# De momento me quedo con el valor del random forest con parámetros personalizados aunque serían necesarias más pruebas. 
# guardo el classification report para el siguiente notebook
report = classification_report(y_test, rf_y_pred_tfidf_cust, output_dict=True)


In [31]:
df_report = pd.DataFrame(report).transpose()

In [32]:
df_report.to_csv('classification_report.csv')

In [33]:
# guardo también la matriz de confusión para el siguiente notebook
df_confusion= pd.crosstab(y_test,rf_y_pred_tfidf_cust)
conf_matrix = df_confusion.to_csv('confusion_matrix.csv')

In [34]:
# saco los datos de la matriz de confusión de True negative, false positive, false negative y true positive
tn, fp, fn, tp = confusion_matrix(y_test, rf_y_pred_tfidf_cust).ravel()
tn, fp, fn, tp

(804, 184, 276, 736)

In [35]:
#también saco los valores de manera individual 
from sklearn import metrics
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test,rf_y_pred_tfidf_cust ))
print("Logistic Regression Precision:",metrics.precision_score(y_test,rf_y_pred_tfidf_cust ))
print("Logistic Regression Recall:",metrics.recall_score(y_test,rf_y_pred_tfidf_cust ))

Logistic Regression Accuracy: 0.77
Logistic Regression Precision: 0.8
Logistic Regression Recall: 0.7272727272727273
